<a href="https://colab.research.google.com/github/Vasan-th/Vasan-th/blob/main/hugging_face_summarization_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
# loading and processing the dataset

from datasets import load_dataset

xsum_data = load_dataset('xsum', streaming = True)
xsum_data

{'train': <datasets.iterable_dataset.IterableDataset at 0x7fe745340a60>,
 'validation': <datasets.iterable_dataset.IterableDataset at 0x7fe745340ee0>,
 'test': <datasets.iterable_dataset.IterableDataset at 0x7fe7452ce100>}

In [ ]:
# sample data
next(iter(xsum_data['train'].take(1)))

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [ ]:
# filtering very short summaries

xsum_data = xsum_data.filter(lambda sample: len(sample['summary']) > 2)
xsum_data

{'train': <datasets.iterable_dataset.IterableDataset at 0x7fe74532af40>,
 'validation': <datasets.iterable_dataset.IterableDataset at 0x7fe7452ce370>,
 'test': <datasets.iterable_dataset.IterableDataset at 0x7fe7444e6250>}

In [ ]:
# building the model instead of using pipeline

from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# preprocessing

max_input_length = 512   # max accepted length of pegasus model
max_target_length = 30   # summary length

def preprocess_function(examples):
  model_inputs = tokenizer(examples['document'], max_length = max_input_length, truncation = True )
  labels = tokenizer(examples['summary'], max_length = max_target_length, truncation = True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
tokenized_dataset = xsum_data.map(preprocess_function, batched = True, remove_columns = ['document', 'summary', 'id'])

In [ ]:
next(iter(tokenized_dataset['train'].take(1))).keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
tokenized_dataset

{'train': <datasets.iterable_dataset.IterableDataset at 0x7fe74725b700>,
 'validation': <datasets.iterable_dataset.IterableDataset at 0x7fe7444e6e50>,
 'test': <datasets.iterable_dataset.IterableDataset at 0x7fe74453ab80>}

In [ ]:
iterable_tokeized_data = iter(tokenized_dataset)
print(next(iterable_tokeized_data))
print(next(iterable_tokeized_data))

train
validation


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint) 

TypeError: ignored

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model = model, return_tensors = 'tf')

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    dataset = tokenized_dataset['train'],
    collate_fn = data_collator,
    shuffle = True,
    batch_size = 8,
    drop_remainder = True,
    prefetch = True
)

tf_valid_dataset = model.prepare_tf_dataset(
    dataset = tokenized_dataset['validation'],
    collate_fn = data_collator,
    shuffle = True,
    batch_size =8,
    drop_remainder = True,
    prefetch = True
)

TypeError: ignored

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

num_train_epochs = 5
num_train_steps = len(tf_train_dataset)*num_train_epochs
model_name = 'pegasus'

optimizer, schedule = create_optimizer(
    init_lr = 5.6e-5,
    num_warmup_steps = 0,
    num_train_steps = num_train_steps,
    weight_decay_rate = 0.01
)

model.compile(optimizer = optimizer)


No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir = f'{model_name}-finetuned-xsum', tokenizer = tokenizer
)

model.fit(tf_train_dataset, validation_data = tf_valid_dataset, callbacks = [callback], epochs = num_train_epochs)

/content/pegasus-finetuned-xsum is already a clone of https://huggingface.co/Vasanth18/pegasus-finetuned-xsum. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/5


In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer


checkpoint = "google/pegasus-xsum"
pipe = pipeline(model = checkpoint)

In [ ]:
from datasets import load_dataset

xsum_data = load_dataset('xsum')

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
%%time 

xsum_data['train']['document'][:3]

CPU times: user 823 ms, sys: 5.99 ms, total: 829 ms
Wall time: 854 ms


['The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - bu

In [ ]:
%%time

xsum_data['train'][:3]['document']        # faster method

CPU times: user 671 µs, sys: 956 µs, total: 1.63 ms
Wall time: 12.5 ms


['The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - bu

In [ ]:
predictions = pipe(xsum_data['train']['document'][:5])       # max sequence length in the dataset is higher than the one model expected

Token indices sequence length is longer than the specified maximum sequence length for this model (1061 > 512). Running this sequence through the model will result in indexing errors


IndexError: ignored

In [ ]:
model_input = tokenizer(xsum_data['train'][:3]['document'], max_length = 512, truncation = True)

In [ ]:
model_input.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
tokenizer.convert_ids_to_tokens(model_input['input_ids'][0])

['▁The',
 '▁full',
 '▁cost',
 '▁of',
 '▁damage',
 '▁in',
 '▁Newton',
 '▁Stewart',
 ',',
 '▁one',
 '▁of',
 '▁the',
 '▁areas',
 '▁worst',
 '▁affected',
 ',',
 '▁is',
 '▁still',
 '▁being',
 '▁assessed',
 '.',
 '▁Repair',
 '▁work',
 '▁is',
 '▁ongoing',
 '▁in',
 '▁Ha',
 'wick',
 '▁and',
 '▁many',
 '▁roads',
 '▁in',
 '▁Peebles',
 'shire',
 '▁remain',
 '▁badly',
 '▁affected',
 '▁by',
 '▁standing',
 '▁water',
 '.',
 '▁Trains',
 '▁on',
 '▁the',
 '▁west',
 '▁coast',
 '▁mainline',
 '▁face',
 '▁disruption',
 '▁due',
 '▁to',
 '▁damage',
 '▁at',
 '▁the',
 '▁La',
 'mington',
 '▁Via',
 'duct',
 '.',
 '▁Many',
 '▁businesses',
 '▁and',
 '▁household',
 'ers',
 '▁were',
 '▁affected',
 '▁by',
 '▁flooding',
 '▁in',
 '▁Newton',
 '▁Stewart',
 '▁after',
 '▁the',
 '▁River',
 '▁Cree',
 '▁overflow',
 'ed',
 '▁into',
 '▁the',
 '▁town',
 '.',
 '▁First',
 '▁Minister',
 '▁Nicola',
 '▁Sturgeon',
 '▁visited',
 '▁the',
 '▁area',
 '▁to',
 '▁inspect',
 '▁the',
 '▁damage',
 '.',
 '▁The',
 '▁waters',
 '▁breached',
 '▁a',
 '

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
